In [1]:
import cv2
from ultralytics import YOLO
import os
import matplotlib.pyplot as plt
import dlib
import numpy as np

proj_path = os.path.abspath(os.path.join(os.getcwd(), '..'))
predictor = dlib.shape_predictor(os.path.join(proj_path, 'data/weights/shape_predictor_68_face_landmarks.dat'))
model = YOLO(os.path.join(proj_path, 'scripts/train_model/train2/weights/best.pt'))

def get_face_feature(img_name):
    img = cv2.imread(os.path.join(proj_path, 'data/photos/' + img_name))
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = model(img_rgb)
    results_boxes = results[0].boxes
    x1, y1, x2, y2 = list(map(int, results_boxes.xyxy[0].tolist()))

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    face = dlib.rectangle(x1, y1, x2, y2)
    landmarks = predictor(gray, face)   
    aligned_face = dlib.get_face_chip(img, landmarks)
    face_recognizer = dlib.face_recognition_model_v1(os.path.join(proj_path, 'data/weights/dlib_face_recognition_resnet_model_v1.dat'))
    face_feature = face_recognizer.compute_face_descriptor(aligned_face)
    print(face_feature.shape)
    return np.array(face_feature).reshape(1, -1), results


In [1]:
import cv2
from ultralytics import YOLO
import os
import matplotlib.pyplot as plt
import dlib
import numpy as np

In [3]:
img = cv2.imread('C:/Users/shann/Desktop/SE/attendance_system/data/photos/biden.jpg')

In [5]:
model = YOLO('C:/Users/shann/Desktop/SE/attendance_system/scripts/train_model/train2/weights/best.pt')

In [7]:
res = model(img)


0: 448x640 1 face, 56.8ms
Speed: 20.5ms preprocess, 56.8ms inference, 325.0ms postprocess per image at shape (1, 3, 448, 640)


In [10]:
res[0].boxes

ultralytics.engine.results.Boxes object with attributes:

cls: tensor([0.], device='cuda:0')
conf: tensor([0.8802], device='cuda:0')
data: tensor([[1.1629e+03, 1.7898e+02, 1.6600e+03, 8.6872e+02, 8.8022e-01, 0.0000e+00]], device='cuda:0')
id: None
is_track: False
orig_shape: (1366, 2048)
shape: torch.Size([1, 6])
xywh: tensor([[1411.4337,  523.8481,  497.1584,  689.7402]], device='cuda:0')
xywhn: tensor([[0.6892, 0.3835, 0.2428, 0.5049]], device='cuda:0')
xyxy: tensor([[1162.8545,  178.9779, 1660.0129,  868.7182]], device='cuda:0')
xyxyn: tensor([[0.5678, 0.1310, 0.8106, 0.6360]], device='cuda:0')

In [2]:
import faiss


index = faiss.IndexFlatIP(128)
index = faiss.IndexIDMap(index)

In [3]:
def get_norm(vec):
    return vec / np.linalg.norm(vec)


In [4]:
ff1 = get_norm(get_face_feature('trump.jpg')[0])
# index.add_with_ids(ff1, np.array([0]))


0: 640x640 1 face, 14.8ms
Speed: 7.0ms preprocess, 14.8ms inference, 182.7ms postprocess per image at shape (1, 3, 640, 640)
(128, 1)


In [5]:

ff2 = get_norm(get_face_feature('trump1.jpg')[0])
dis2, _ = index.search(ff2, 1)
ff3 = get_norm(get_face_feature('trump2.jpg')[0])
dis3, _ = index.search(ff3, 1)
ff4 = get_norm(get_face_feature('girl-1.jpg')[0])
dis4, _ = index.search(ff4, 1)


0: 640x640 1 face, 16.0ms
Speed: 4.0ms preprocess, 16.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

0: 448x640 1 face, 49.0ms
Speed: 2.0ms preprocess, 49.0ms inference, 2.0ms postprocess per image at shape (1, 3, 448, 640)

0: 640x448 1 face, 57.0ms
Speed: 4.0ms preprocess, 57.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 448)


In [6]:
print(dis2, dis3, dis4)

[[    0.92361]] [[    0.93602]] [[    0.83565]]


c:\Users\shann\anaconda3\envs\SE\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
model = InceptionResnetV1(pretrained='vggface2').eval().to('cuda')


In [ ]:
import cv2
from ultralytics import YOLO
import os
import matplotlib.pyplot as plt
import dlib
import numpy as np

proj_path = os.path.abspath(os.path.join(os.getcwd(), '..'))
predictor = dlib.shape_predictor(os.path.join(proj_path, 'data/weights/shape_predictor_68_face_landmarks.dat'))
yolo_model = YOLO(os.path.join(proj_path, 'scripts/train_model/train2/weights/best.pt'))

img = cv2.imread(os.path.join(proj_path, 'data/photos/' + 'trump.jpg'))
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
results = yolo_model(img_rgb)
results_boxes = results[0].boxes
x1, y1, x2, y2 = list(map(int, results_boxes.xyxy[0].tolist()))





0: 640x640 1 face, 77.8ms
Speed: 8.0ms preprocess, 77.8ms inference, 9.9ms postprocess per image at shape (1, 3, 640, 640)


In [15]:
face = img_rgb[x1:x2, y1:y2]
feature = model(torch.from_numpy(face).permute(2, 0, 1).unsqueeze(0).float().to('cuda'))

In [25]:
feature = feature.cpu().detach().numpy()


In [30]:
feature = feature / np.linalg.norm(feature)
print(np.min(feature))

-0.12359458
